Using this to write out my resume, then copying it. 

Would be a fun project to do this automatically, now that I have all the styling etc. for it :) 

# Daniel McCann-Sayles' Portfolio

### Summary

<p style="display: flex; justify-content: space-between; right-margin: 50px ">
  <span>Cisco</span>
  <span ">thing2</span>
</p>

Summary :p

### Work Experience

Cisco - Intern

### Projects

##### Notebook portfolio

ongoing...

##### Kid Fight Game

##### MACM exploration

##### AI Generation

##### Speech to LLM App


##### Cisco Hackathon

I competed in an internal Cisco hackathon to explore how we could improve  customer experience use cases with AI. 

My team won 1st place out of over 20 teams!

The compeotition took place over the span of two weeks. It was hosted by a different organization in Cisco, so it was different products and a different setting than I'm used to. Luckily my team had some experience we were able to draw on.

I had a lot of fun meeting people, collaborating, and creating our project. 


I played a few key roles on the team: 
1. Collaborating closely with our team lead to hash out his ideas and customer knowledge, and come up with a winning idea. He has a lot of knowledge around the specific field; *together* we were able to turn this into an implementation. 
2. Executing on this; I quickly learned how to build, and built, a smart AI assistant using Streamlit and a RAG system with FAISS and LangChain.

This showcases my holistic skill set. <span id="text-accent">I’m at my best when I have the ability to work closely with the product, with stakeholders, and with the implementation itself!</span>

My team lead graciously said this about me:

> "Daniel continuously demonstrated his ability to navigate high-level business strategy and translate that into ideas, concepts, code, and connections that support the achievement of those goals."

** add some more about this showing my ideation, prototyping, and ability to see the big picture, while also coding it! **
